In [ ]:
import findspark
findspark.init('/home/peyman/spark-2.0.0-bin-hadoop2.7')

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'


import sys
import time
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from operator import add


n_secs = 30
topic = "pysparkKafkaProject"

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[*]")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'localhost:9092', 
                        'group.id':'video-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
    
lines = kafkaStream.map(lambda x: x[1])
ip_pairs = lines.map(lambda x:(x.split('-')[0].strip(),1))
ip_counts = ip_pairs.reduceByKey(add)
potential_attackers = ip_counts.filter(lambda x:x[1]>1).map(lambda x:x[0])
potential_attackers.pprint()


ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)    

-------------------------------------------
Time: 2018-12-31 02:38:30
-------------------------------------------

-------------------------------------------
Time: 2018-12-31 02:39:00
-------------------------------------------
64.242.88.10

-------------------------------------------
Time: 2018-12-31 02:39:30
-------------------------------------------

-------------------------------------------
Time: 2018-12-31 02:40:00
-------------------------------------------
64.242.88.10

